# SPMV: sparse-matrix (dense)-vector multiplication

Let's look the csr (compressed sparse row) format for storing sparse matrices.

A dense matrix is converted into a csr-formatted sparse matrix in the following way:

1. Collect all nonzero entries into a 1-D "data" array, sorted first by row, and then by column. Equivalently, concatenate the rows of the matrix and extract the ordered list of nonzero entries.
2. Enter their corresponding column indices into another 1-D "indices" array of the same size as and in one-to-one correspondence with "data". E.g. indices[4] is the column index of the entry in data[4].
3. Create a final 1-D array "indptr" indicating which entries in data/indices correspond to which rows. E.g. if row 5 has 3 nonzero entries in the dense matrix, and these are entries 7, 8, and 9 in "data", we would have indptr[5] = 7 and indptr[6] = 10, specifying that these entries and their corresponding column indices are listed in order at positions 7 - 10 (upper bound exclusive, a la python) in data and indices, respectively.

### Example

In [1]:
%reset -f
import numpy as np
from scipy.sparse import csr_matrix

dense = np.array([
    [0, 0, 0, 1, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0],
    [0, 0, 4, 0, 0, 0, 3],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 5, 0, 0, 0],
    [1, 0, 0, 2, 4, 0, 0],
    [0, 3, 0, 0, 0, 0, 0],
], dtype=np.float32)

sparse = csr_matrix(dense)

print('sparse.data: ')
print(sparse.data)
print('')
print('sparse.indices: ')
print(sparse.indices)
print('')
print('sparse.indptr: ')
print(sparse.indptr)
print('')

sparse.data: 
[ 1.  2.  4.  3.  5.  1.  2.  4.  3.]

sparse.indices: 
[3 1 2 6 3 0 3 4 1]

sparse.indptr: 
[0 1 2 4 4 5 8 9]



# Using pyculib.sparse

In [ ]:
%reset -f
import numpy as np
from pyculib.sparse import Sparse
from pyculib.sparse.binding import CUSPARSE_OPERATION_NON_TRANSPOSE
from scipy.sparse import csr_matrix


def spmv_cuda(a_sparse, b, sp):
    """Compute a_sparse @ b using C"""

    # args to csrmm call
    trans_a = 'N'  # non-transpose, use 'T' for transpose or 'C' for conjugate transpose
    m = a_sparse.shape[0]  # num rows in a
    n = b.shape[1]  # num cols in b, c
    k = a_sparse.shape[1]  # num cols in a
    nnz = len(a_sparse.data)  # num nonzero in a
    alpha = 1
    descr_a = sp.matdescr(  # matrix descriptor
        indexbase=0,  # 0-based indexing
        matrixtype='G',  # 'general': no symmetry or triangular structure
    )
    csr_val_a = a_sparse.data
    csr_row_ptr_a = a_sparse.indptr
    csr_col_ind_a = a_sparse.indices
    ldb = b.shape[0]
    beta = 0
    c = np.empty(b.shape, dtype=np.float32)
    ldc = b.shape[0]

    # call function
    sp.csrmm(
        transA=trans_a,
        m=m,
        n=n,
        k=k,
        nnz=nnz,
        alpha=alpha,
        descrA=descr_a,
        csrValA=csr_val_a,
        csrRowPtrA=csr_row_ptr_a,
        csrColIndA=csr_col_ind_a,
        B=b,
        ldb=ldb,
        beta=beta,
        C=c,
        ldc=ldc)
    
    return c


a_dense = np.array([
    [0, 0, 0, 1, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0],
    [0, 0, 4, 0, 0, 0, 3],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 5, 0, 0, 0],
    [1, 0, 0, 2, 4, 0, 0],
    [0, 3, 0, 0, 0, 0, 0],
], dtype=np.float32)

b = np.array([[1, 2, 3, 4, 5, 6, 7]], dtype=np.float32).T

c_correct = np.array([[4, 4, 33, 0, 20, 29, 6]], dtype=np.float32).T

a_sparse = csr_matrix(a_dense)

c = spmv_cuda(a_sparse, b, Sparse())

print('transpose of computed C: ')
print(c.T)
print('')
print('transpose of correct C: ')
print(c_correct.T)

assert np.allclose(c, c_correct)

Benchmarks for large matrices:

In [ ]:
from time import time

COUNT = 1
N = 10000
P = 0.05

print('Constructing objects...')
np.random.seed(0)
a_dense = (np.random.rand(N, N) < P).astype(float)
a_sparse = csr_matrix(a_dense)

b = np.random.rand(N)

sp = Sparse()

print('Testing scipy sparse matrix multiplication...\n')
tic = time()
for ii in range(COUNT):
    c = a_sparse.dot(b)
toc = time()

print('scipy matrix multiplication took {} seconds\n\n'.format(toc - tic))

print('Testing pyculib sparse matrix multiplication...\n')

tic = time()
for ii in range(COUNT):
    c = spmv_cuda(a_sparse, b, sp)
toc = time()

print('pyculib matrix multiplication took {} seconds\n\n'.format(toc - tic))